In [1]:
# Start writing code here...
import numpy as np 
import pandas as pd

pd.options.mode.chained_assignment = None 

import os, re
import json
import matplotlib.pyplot as plt
  
from IPython.display import display, clear_output
    
from tqdm import tqdm
import string

from sklearn.model_selection import train_test_split

os.listdir('/kaggle/input/coleridgeinitiative-show-us-the-data/')

['sample_submission.csv', 'train.csv', 'test', 'train']

In [2]:
validation = pd.read_csv("../input/colerigde-processed-text/validation.csv",index_col=0)
test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [3]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6255431350680110528,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15685569792
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3157770517079396094
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [4]:
!pip install --no-index --find-links "../input/spacy3" spacy[cuda110]
!pip install --no-index --find-links "../input/spacy3" en_core_web_trf
!pip install --no-index --find-links "../input/spacy3" spacy_transformers

Looking in links: ../input/spacy3
Looking in links: ../input/spacy3
Processing /kaggle/input/spacy3/en_core_web_trf-3.0.0-py3-none-any.whl
Processing /kaggle/input/spacy3/spacy-3.0.6-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/spacy3/spacy_transformers-1.0.2-py2.py3-none-any.whl
Processing /kaggle/input/spacy3/spacy_legacy-3.0.5-py2.py3-none-any.whl
Processing /kaggle/input/spacy3/pydantic-1.7.3-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/spacy3/catalogue-2.0.3-py3-none-any.whl
Processing /kaggle/input/spacy3/thinc-8.0.3-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/spacy3/pathy-0.5.2-py3-none-any.whl
Processing /kaggle/input/spacy3/srsly-2.4.1-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/spacy3/typer-0.3.2-py3-none-any.whl
Processing /kaggle/input/spacy3/smart_open-3.0.0-py3-none-any.whl
Processing /kaggle/input/spacy3/spacy_alignments-0.8.3-cp37-cp37m-manylinux2014_x86_64.whl
  Attempting uninstall: catalogue
    Found 

In [5]:
import spacy
from spacy.tokens import DocBin

In [6]:
print(spacy.__version__)

3.0.6


In [7]:
# Load the best model
try:
    spacy.require_gpu()
except:
    print("GPU not found")
    
nlp = spacy.load(R"../input/coledridge-challenge-models/model-best")

In [8]:
# Testing the model
doc = nlp('The supply of PCR reagents, trained lab personnel and the availability of laboratories with sufficient biocontainment levels are major challenges of SARS-CoV-2 detection in developing countries, such as Indonesia (Younes et al., 2020) . Therefore, it is not surprising that the tested people per week is still lower than the World Health Organization (WHO) standard (World Health Organization, 2020b) . Recently, SARS-CoV-2 with the D614G mutation became the most frequently detected globally, including South East Asia region (Korber et al., 2020; Nguyen et al., 2020) . Interestingly, SARS-CoV-2 with the G614 variant had significantly higher infectious titers than the original D614 virus, and COVID-19 patients with the G614 variant had a higher viral load than patients without the mutation (Korber et al., 2020) . A recent study showed that the SARS-CoV-2 with the G614 variant revealed increased infectivity, competitive fitness, and transmission than the wild-type D614 virus in human airway epithelial cells and hamster (Hou et al., 2020) . However, this mutation was not associated with the severity of COVID-19 (Korber et al., 2020; Nguyen et al., 2020) . Here, we aimed:\n(1) to report full-length genome sequences of SARS-CoV-2 collected from four COVID-19 patients in the Special Region of Yogyakarta and Central Java provinces, Indonesia;\n(2) to compare the clade distribution of full-length genome sequences from Indonesia (n = 60) from March to September 2020; and (3) to perform phylogenetic analysis of SARS-CoV-2 complete genomes from different countries, including Indonesia.') # input sample text
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('genome sequences of SARS-CoV-2', 'DATASET')]


In [9]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [10]:
validation.head()

,Section,Text,Label,Id,dataset_index
0,Abstract,A rterial stiffness is an age-related trait th...,Baltimore Longitudinal Study of Aging,58147ea8-c5c7-4283-bb27-c01830c3af13,"[(1309, 1346, 'DATASET'), (2525, 2563, 'DATASE..."
1,Introduction,"(2012 Jedynak et al. ( , 2015 proposed a param...",Alzheimer's Disease Neuroimaging Initiative (A...,1c35d627-8b39-41d2-b3e0-77c92483d8a8,"[(1925, 1975, 'DATASET')]"
2,Impact of Intensification Rate on TC Intensity...,[11] How do the changes in intensification rat...,IBTrACS,5716fd22-6c6c-4587-a673-281a33456749,"[(334, 341, 'DATASET')]"
3,Introduction,Sponsored by the National Center for Education...,Early Childhood Longitudinal Study,3be437b7-2576-4b2f-ae22-d158cf97edfe,"[(59, 93, 'DATASET')]"
4,Introduction,"(2010) , we developed a novel surface mapping ...",Alzheimer's Disease Neuroimaging Initiative (A...,6b460753-749f-45bc-9996-dd8a950107ce,"[(1787, 1837, 'DATASET')]"


In [11]:
# Testing the model
doc = nlp("investigated the differences in college access and choice among students of different racialethnic groups using the national educational longitudinal study nels8892 and the beginning postsecondary student longitudinal study bps9092") # input sample text
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('beginning postsecondary student', 'DATASET')]


In [12]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [13]:
submission = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')

submission.head()

,Id,PredictionString
0,2100032a-7c33-4bff-97ef-690822c43466,NaN
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN


### Validate model

In [14]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

In [15]:
def extract_ds_from_text(path):
    extract_df = pd.read_json(path)
    
    dataset = []

    for idx, r in extract_df.iterrows():
        doc = nlp(r['text'])

        if doc.ents:
            for i in [ent.text for ent in doc.ents]:
                i = clean_text(i)
                if len(i) != 0 and i not in dataset:
                    dataset.append(str(i).strip())

    return dataset

In [16]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [17]:
jaccard("national education longitudinal study","education longitudinal study")

0.75

In [18]:
for index, rows in tqdm(validation.iterrows()):
    doc = nlp(rows['Text'])

    preds = []
    
    if doc.ents:
        for i in [ent.text for ent in doc.ents]:
            i = clean_text(i)
            if len(i) != 0:
                preds.append(str(i).strip())
    
    validation.loc[index, 'predictions'] = '|'.join(preds)

2854it [02:47, 17.02it/s]


In [19]:
validation['Label'] = validation['Label'].apply(clean_text)

In [20]:
validation['truth'] = validation.groupby(['Id'])['Label'].transform(lambda x : '|'.join(x))
validation['predictions'] = validation.groupby(['Id'])['predictions'].transform(lambda x : '|'.join(x))
validation = validation.drop_duplicates("Id")

In [21]:
validation = validation[['Id', 'truth', 'predictions']]

In [22]:
validation.head(10)

,Id,truth,predictions
0,58147ea8-c5c7-4283-bb27-c01830c3af13,baltimore longitudinal study of aging|baltimor...,baltimore longitudinal study of aging|baltimor...
1,1c35d627-8b39-41d2-b3e0-77c92483d8a8,alzheimer s disease neuroimaging initiative adni,alzheimer s disease neuroimaging initiative ad...
2,5716fd22-6c6c-4587-a673-281a33456749,ibtracs|ibtracs,ibtracs|ibtracs|ibtracs|ibtracs|ibtracs|ibtrac...
3,3be437b7-2576-4b2f-ae22-d158cf97edfe,early childhood longitudinal study,early childhood longitudinal study
4,6b460753-749f-45bc-9996-dd8a950107ce,alzheimer s disease neuroimaging initiative ad...,alzheimer s disease neuroimaging initiative ad...
5,e13db490-cf95-4bda-b730-5376b67a8a99,north american breeding bird survey,north american breeding bird survey
6,8435368a-eef2-4438-8f65-f78d0d888f68,adni,adni
7,d3dc9bd8-e2cc-47ed-bf03-03f37323b09c,baltimore longitudinal study of aging|baltimor...,baltimore longitudinal study of aging|baltimor...
8,235872d8-9b25-492e-8e74-a3faae37978a,adni,alzheimer s disease neuroimaging initiative adni
9,93572da5-bcb2-4f91-a963-d35f6c528b03,national education longitudinal study,education longitudinal study


In [23]:
TP = 0

FP = 0
FN = 0

for index, rows in validation.iterrows():
    predictions = np.array(str(rows['predictions']).split('|'))
    ground_truths = np.array(str(rows['truth']).split('|'))
    
    predictions = np.unique(predictions)
    ground_truths = np.unique(ground_truths)
    
    predictions = np.sort(predictions)

    for pred in predictions:
        for truth in ground_truths:
            if pred in truth:
                if jaccard(pred,truth) >= 0.5:
                    TP += 1
                else:
                    FP += 1
            else:
                FP += 1
                
            if truth not in predictions:
                FN += 1

In [24]:
print("True positives: {}".format(TP))
print("\n")
print("False positives: {}".format(FP))
print("False negatives: {}".format(FN))

True positives: 2138


False positives: 939
False negatives: 1053


In [25]:
beta = 0.5
precision = (TP/(TP+FP))
recall = (TP/(TP+FN))

fbeta = ((1 + beta**2) * precision * recall) / (beta**2 * precision + recall)

print("F0.5 score: {}".format(fbeta))

F0.5 score: 0.6897219175430672


### Create Submissions file

In [26]:
example = "../input/coleridgeinitiative-show-us-the-data/test/2f392438-e215-4169-bebf-21ac4ff253e1.json"

print(extract_ds_from_text(example))

['trends in international mathematics and science study']


In [27]:
for index, rows in submission.iterrows():
    preds = extract_ds_from_text((test_files_path+"/"+rows["Id"]+".json"))
    submission.loc[index, 'PredictionString'] = '|'.join(preds)

In [28]:
submission.head()

,Id,PredictionString
0,2100032a-7c33-4bff-97ef-690822c43466,alzheimer s disease neuroimaging initiative adni
1,2f392438-e215-4169-bebf-21ac4ff253e1,trends in international mathematics and scienc...
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,slosh model
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,rural urban continuum codes


In [29]:
submission.to_csv('submission.csv', index=False)